In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

# Load API Credentials
with open('/Users/youngjinjang/Desktop/dojo-env-setup/docs/Yelp assignment/Yelp/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
    
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [2]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [3]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Sushi'

In [4]:
# Directory to save results from api calls
JSON_FILE = "data/results_in_progress_NY_sushi.json"
JSON_FILE

'data/results_in_progress_NY_sushi.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] data/results_in_progress_NY_sushi.json already exists.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 60 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

4000

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

197

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [12]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/197 [00:00<?, ?it/s]

In [15]:
for i in tqdm_notebook(range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
        
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/197 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [16]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,BPZfuPvSxCfoppU4lPFm0Q,domodomo-new-york-new-york-4,DOMODOMO - New York,https://s3-media1.fl.yelpcdn.com/bphoto/YX89PD...,False,https://www.yelp.com/biz/domodomo-new-york-new...,1462,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 40.7280299, 'longitude': -74.00167}",[delivery],$$$,"{'address1': '140 W Houston St', 'address2': N...",+16467070301,(646) 707-0301,2584.282555
1,pNe8ZCkGTe4cXfKdOF7kYQ,ise-restaurant-new-york-3,Ise Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/S5SGXw...,False,https://www.yelp.com/biz/ise-restaurant-new-yo...,591,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 40.7291024, 'longitude': -73.9900...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '63 Cooper Sq', 'address2': None,...",+12122284152,(212) 228-4152,2654.657530
2,pGYovGGnxm0tMmkiQJPygA,ume-williamsburg-brooklyn,Ume Williamsburg,https://s3-media1.fl.yelpcdn.com/bphoto/Xf4Yxw...,False,https://www.yelp.com/biz/ume-williamsburg-broo...,381,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.0,"{'latitude': 40.71657, 'longitude': -73.96559}",[delivery],$$$$,"{'address1': '237 Kent Ave', 'address2': None,...",+19294203253,(929) 420-3253,2724.422854
3,pou1-d7PENs6AD4mnt9P4g,sushi-w-new-york,Sushi W,https://s3-media3.fl.yelpcdn.com/bphoto/hxguGJ...,False,https://www.yelp.com/biz/sushi-w-new-york?adju...,112,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 40.79837, 'longitude': -73.96941}",[],NaN,"{'address1': '2673 Broadway', 'address2': '', ...",+19173189739,(917) 318-9739,10552.517781
4,PjjpgjY_sdawJU1JHbyNTQ,temakase-new-york,Temakase,https://s3-media3.fl.yelpcdn.com/bphoto/mPHfCH...,False,https://www.yelp.com/biz/temakase-new-york?adj...,511,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 40.729711, 'longitude': -73.987127}","[pickup, delivery]",NaN,"{'address1': '157 Second Ave', 'address2': Non...",+12123811456,(212) 381-1456,2766.791755


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,5NZK0ImHGPmSLVyjCYaNPA,sushi-you-bloomfield,Sushi You,https://s3-media1.fl.yelpcdn.com/bphoto/bVCCcw...,False,https://www.yelp.com/biz/sushi-you-bloomfield?...,184,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 40.83662, 'longitude': -74.1641413}","[pickup, delivery]",$$,"{'address1': '397 W Passaic Ave', 'address2': ...",+19733380899,(973) 338-0899,20413.223089
996,TuoXbiDqhFBJiyAXpcOLuA,genji-sushi-kano-new-york,Genji Sushi Kano,https://s3-media1.fl.yelpcdn.com/bphoto/o4eOJo...,False,https://www.yelp.com/biz/genji-sushi-kano-new-...,9,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 40.75465, 'longitude': -73.98476}",[],$$,"{'address1': '1095 6th Ave', 'address2': None,...",+19177285700,(917) 728-5700,5536.142601
997,ZSzJiWKmcf2KSgx1_MN0lg,chakra-japanese-fusion-flushing-2,Chakra Japanese Fusion,https://s3-media4.fl.yelpcdn.com/bphoto/Rq-jMm...,False,https://www.yelp.com/biz/chakra-japanese-fusio...,59,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...",4.0,"{'latitude': 40.743631, 'longitude': -73.825962}","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '59-08 Main St', 'address2': None...",+17187990793,(718) 799-0793,14809.353237
998,dC2VDnFveu7JPNY_X9NvxQ,poke-bowl-new-york-13,Poke Bowl,https://s3-media4.fl.yelpcdn.com/bphoto/Ln5S5L...,False,https://www.yelp.com/biz/poke-bowl-new-york-13...,56,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",4.0,"{'latitude': 40.7372124400428, 'longitude': -7...","[pickup, delivery]",NaN,"{'address1': '26 East 17th St', 'address2': No...",,,3548.510782
999,BXCkH12ezgebuMRnW-6Ung,las-vegas-sushi-fort-lee,Las Vegas Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/BgB81P...,False,https://www.yelp.com/biz/las-vegas-sushi-fort-...,195,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 40.85319496760645, 'longitude': -...","[pickup, delivery]",$$$,"{'address1': '1475 Bergen Blvd', 'address2': '...",+12014820870,(201) 482-0870,16445.151868
